# MIR100 Graph

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors
from matplotlib.cm import Blues

import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.lines import Line2D 

In [2]:
def parse_strategies_from_log(log_file):
    """Extract defense strategies and attack paths from the log file"""
    defense_strategy = {}
    attack_paths = []
    attack_probs = []
    attack_rate = None  # Initialize to None
    defense_rate = None # Initialize to None
    attacker_success = 0.0 # Initialize to float

    try:
        with open(log_file, 'r') as f:
            log_content = f.read()

        # Parse attack paths
        if 'Debug - Current Attack paths:' in log_content:
            attack_paths_section = log_content.split('Debug - Current Attack paths:')[1].split('++++++')[0].strip()
            for line in attack_paths_section.strip().split('\n'):
                if ':' in line:
                    path_index, path_str = line.strip().split(':', 1)
                    attack_paths.append(path_str.strip())

        # Parse attack and defense rates
        if 'attack rate =' in log_content:
            # Split based on the first occurrence of 'attack rate ='
            rate_line = log_content.split('attack rate =', 1)[1].split('\n')[0]
            parts = rate_line.split(',') # Split the remaining part by comma
            
            # Parse attack rate
            attack_rate_str = parts[0].strip()
            if attack_rate_str.lower() == 'none':
                attack_rate = None
            else:
                try:
                    attack_rate = int(attack_rate_str)
                except ValueError:
                    print(f"Warning: Could not parse attack rate '{attack_rate_str}' as integer.")
                    attack_rate = None # Or handle the error as needed

            # Parse defense rate (assuming it follows the comma)
            if len(parts) > 1 and 'defense rate =' in parts[1]:
                 defense_rate_str = parts[1].split('defense rate =')[1].strip()
                 if defense_rate_str.lower() == 'none':
                     defense_rate = None
                 else:
                     try:
                         defense_rate = int(defense_rate_str)
                     except ValueError:
                         print(f"Warning: Could not parse defense rate '{defense_rate_str}' as integer.")
                         defense_rate = None # Or handle the error as needed

        # Parse defense strategy
        if 'optimal defense strategy:' in log_content:
            defense_section = log_content.split('optimal defense strategy:')[1].split('worst case attack strategies')[0]
            for line in defense_section.strip().split('\n')[1:]:  # Skip header
                if line.strip():
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        try:
                            node_id = int(parts[0])
                            probability = float(parts[1])
                            defense_strategy[node_id] = probability
                        except (ValueError, IndexError):
                             print(f"Warning: Could not parse defense strategy line: '{line.strip()}'")


        # Parse attack strategy probabilities
        if 'worst case attack strategies per goal:' in log_content:
             # Split based on the first occurrence
            attack_section = log_content.split('worst case attack strategies per goal:', 1)[1].split('[1]')[0] # Assuming [1] marks the end or next section reliably
            for line in attack_section.strip().split('\n')[1:]:  # Skip header
                if line.strip():
                    parts = line.strip().split()
                    if len(parts) >= 2:
                         try:
                            prob = float(parts[1])
                            attack_probs.append(prob)
                         except (ValueError, IndexError):
                            print(f"Warning: Could not parse attack probability line: '{line.strip()}'")


        # Parse attacker success probability
        if 'Attacker can guarantee success probability of:' in log_content:
             # Split based on the first occurrence
            success_line = log_content.split('Attacker can guarantee success probability of:', 1)[1].strip().split('\n')[0] # Take only the first line after the marker
            try:
                attacker_success = float(success_line.split()[0])
            except (ValueError, IndexError):
                 print(f"Warning: Could not parse attacker success probability from '{success_line}'")
                 attacker_success = 0.0 # Default or handle error

        return defense_strategy, attack_paths, attack_probs, attack_rate, defense_rate, attacker_success

    except FileNotFoundError:
        print(f"Error: Log file '{log_file}' not found.")
        return {}, [], [], None, None, 0.0
    except Exception as e:
        print(f"Error parsing log file '{log_file}': {e}")
        # Return defaults or re-raise depending on desired error handling
        return {}, [], [], None, None, 0.0

In [3]:
def defense_color_mapping(probability, min_prob=0.05, max_prob=1.0):
    """Map defense probability to a blue gradient color"""
    if max_prob == min_prob:
        normalized = 1.0  
    else:
        normalized = (probability - min_prob) / (max_prob - min_prob)
    
    color_value = 0.65 + (normalized * 0.4)
    return Blues(color_value)

In [4]:
def graph_plotting(basename): # Added attack_graph as parameter
    """Enhanced graph plotting with strategy information from log file"""

    # Parse strategies from log file (Keep this for now, as requested, but it might color over green/red)
    defense_strategy, attack_paths, attack_probs, attack_rate, defense_rate, attacker_success = parse_strategies_from_log(f"{basename}.log")

    # Filter defense strategies with probabilities < 5%
    filtered_defense = {node: prob for node, prob in defense_strategy.items() if prob >= 0.05}

    # Find min and max defense probabilities for gradient scaling
    if filtered_defense:
        min_prob = min(filtered_defense.values())
        max_prob = max(filtered_defense.values())
    else:
        min_prob, max_prob = 0.05, 1.0 # Default range if no defense info above threshold

    # Create a basic spring layout as starting point (Keep layout logic as is for now)
    pos = nx.spring_layout(attack_graph, k=1, iterations=50, seed=42)

    # --- Start of Changed Lines ---

    # Hardcode entry and target nodes as requested
    entry_nodes = [1, 2, 3, 4]  # Nodes 1, 2, 3, 4 are entry (green)
    target_nodes = [12, 13, 14, 16] # Nodes 12, 13, 14, 16 are target (red)
    
    # --- End of Changed Lines ---


    # Identify intermediate nodes (using the hardcoded entry/target nodes)
    all_nodes = list(attack_graph.nodes())
    # Note: Using the first entry node for path calculation for layout.
    reference_node_for_layout = entry_nodes[0] if entry_nodes else list(attack_graph.nodes())[0] 
    
    intermediate_nodes = [node for node in all_nodes if node not in entry_nodes and node not in target_nodes]

    # Calculate node distances from the reference node (shortest path length)
    distances = {}
    for node in all_nodes:
        try:
            # Calculate distance from the first entry node (Node 1 in this hardcoded case)
            distances[node] = nx.shortest_path_length(attack_graph, reference_node_for_layout, node) 
        except nx.NetworkXNoPath:
            # No path from reference to this node, set a large distance
            distances[node] = len(attack_graph) * 2 

    # Normalize distances to determine y-coordinate
    max_dist = max(distances.values()) if distances else 1

    # Set y-coordinate based on distance from reference node
    for node in all_nodes:
         if node in pos:
            dist = distances.get(node, max_dist) 
            pos[node][1] = 1.0 - (dist / max_dist if max_dist > 0 else 0)

    # Ensure entry nodes are near the top and target nodes near the bottom
    # Adjust y-coordinates based on the hardcoded lists
    for node in entry_nodes:
        if node in pos:
            pos[node][1] = 1.0 # Place entry nodes at the top
    for node in target_nodes:
        if node in pos:
            pos[node][1] = 0.0 # Place target nodes at the bottom


# --- Start: Updated Manual Position Adjustments to Match Image ---
    # Top Layer (Nodes 3, 4, 2, 1) ~ y=1.0
    if 3 in pos: pos[3][0] = -0.8; pos[3][1] = 1.0  # Node 3: Far left top
    if 4 in pos: pos[4][0] = -0.4; pos[4][1] = 1.0  # Node 4: Mid-left top
    if 2 in pos: pos[2][0] =  0.4; pos[2][1] = 1.0  # Node 2: Mid-right top
    if 1 in pos: pos[1][0] =  0.8; pos[1][1] = 1.0  # Node 1: Far right top

    # Second Layer (Nodes 6, 8) ~ y=0.8
    if 6 in pos: pos[6][0] = -0.8; pos[6][1] = 0.8  # Node 6: Far left upper-mid
    if 8 in pos: pos[8][0] = -0.4; pos[8][1] = 0.8  # Node 8: Mid-left upper-mid

    # Third Layer (Node 7) ~ y=0.6
    if 7 in pos: pos[7][0] =  0.0; pos[7][1] = 0.8  # Node 7: Center mid

    # Fourth Layer (Nodes 10, 9, 11, 5) ~ y=0.4
    if 10 in pos: pos[10][0] = -0.8; pos[10][1] = 0.4 # Node 10: Mid-left lower-mid
    if 9 in pos: pos[9][0] =  0.2; pos[9][1] = 0.4  # Node 9: Center lower-mid
    if 11 in pos: pos[11][0] = 0.4; pos[11][1] = 0.4 # Node 11: Mid-right lower-mid
    if 5 in pos: pos[5][0] =  0.8; pos[5][1] = 0.4  # Node 5: Far right lower-mid

    # Fifth Layer (Node 15) ~ y=0.2
    if 15 in pos: pos[15][0] = 0.2; pos[15][1] = 0.2 # Node 15: Slightly right, above bottom

    # Bottom Layer (Nodes 14, 16, 12, 13) ~ y=0.0
    if 14 in pos: pos[14][0] = -0.75; pos[14][1] = 0.0 # Node 14: Far left bottom
    if 16 in pos: pos[16][0] = -0.25; pos[16][1] = 0.0 # Node 16: Mid-left bottom
    if 12 in pos: pos[12][0] =  0.25; pos[12][1] = 0.0 # Node 12: Mid-right bottom
    if 13 in pos: pos[13][0] =  0.75; pos[13][1] = 0.0 # Node 13: Far right bottom
    # --- End: Updated Manual Position Adjustments ---

    # Create figure with GridSpec for better control of layout
    fig = plt.figure(figsize=(10, 8))
    gs = fig.add_gridspec(2, 1, height_ratios=[3, 1])  # 2 rows: graph and legends
    
    # Main network plot area
    ax_graph = fig.add_subplot(gs[0])
    
    # Draw the network in the main area
    nx.draw_networkx_edges(attack_graph, pos, ax=ax_graph, edge_color='gray', arrows=True, arrowsize=20, width=2)

    # --- Coloring Logic (uses the hardcoded lists now) ---
    node_colors = []
    for node in attack_graph.nodes():
        if node in entry_nodes: # Check if node is in the hardcoded entry list [1, 2, 3, 4]
            node_colors.append('lightgreen')
        elif node in target_nodes: # Check if node is in the hardcoded target list [12, 13, 14, 16]
            node_colors.append('lightcoral')
        else:
            node_colors.append('lightblue') # Default for intermediate nodes
    # --- End of Coloring Logic ---

    # --- This section overwrites colors based on log file - keep as requested ---
    white_text_nodes = []
    node_list = list(attack_graph.nodes())
    # Be careful: If an entry/target node also has a defense probability > 5%, 
    # its color will be overridden by the blue gradient below.
    for node, prob in filtered_defense.items():
        if node in node_list:
            try:
                idx = node_list.index(node)
                node_colors[idx] = defense_color_mapping(prob, min_prob, max_prob)
                if prob > (min_prob + (max_prob - min_prob) * 0.5): 
                     white_text_nodes.append(node)
            except IndexError:
                 print(f"Warning: Node {node} from defense strategy not found in node_list index.")
            except ValueError:
                 print(f"Warning: Node {node} from defense strategy not found in node_list.")
    # --- End of color overwriting section ---

    nx.draw_networkx_nodes(attack_graph, pos, ax=ax_graph, node_color=node_colors,  
                           node_size=700, edgecolors='black', linewidths=2)

    node_labels = {node: str(node) for node in attack_graph.nodes()}
    nx.draw_networkx_labels(attack_graph, pos, ax=ax_graph, labels=node_labels,  
                            font_size=12, font_weight='bold', font_color='black')
    
    white_labels = {node: str(node) for node in white_text_nodes if node in node_labels}
    if white_labels:
        nx.draw_networkx_labels(attack_graph, pos, ax=ax_graph, labels=white_labels,  
                                font_size=12, font_weight='bold', font_color='white')

    # Turn off axis for graph
    ax_graph.axis('off')

    # --- Legend Section (Keep as is for now, relies on log file parsing) ---
    if legends:
        ax_graph.set_title("Optimal Strategies (≥5%)", fontsize=12, fontweight='bold', pad=10)
        ax_legends = fig.add_subplot(gs[1])
        ax_legends.axis('off')  
        
        col1_elements = [
            plt.Line2D([0],[0], linestyle="None", marker="", label=f'Attack rate: {attack_rate}'),
            plt.Line2D([0],[0], linestyle="None", marker="", label=f'Defense rate: {defense_rate}'),
            plt.Line2D([0],[0], linestyle="None", marker="", label=f'Attacker success: {attacker_success:.4f}')
        ]

        sorted_defense = sorted(filtered_defense.items(), key=lambda item: item[1], reverse=True)
        col2_elements = [
            plt.Line2D([0],[0], linestyle="None", marker='o', 
                       markerfacecolor=defense_color_mapping(prob, min_prob, max_prob), markersize=8, 
                       label=f'Defense: Node {node} ({prob:.2f})')  
            for node, prob in sorted_defense
        ]

        valid_attack_strategies = [(path, prob) for path, prob in zip(attack_paths, attack_probs) if prob >= 0.05]
        col3_elements = [
             plt.Line2D([0],[0], linestyle="-", color='black', linewidth=1.5, 
                       label=f'Attack path {i+1}: {path} ({prob:.2f})')  
             for i, (path, prob) in enumerate(valid_attack_strategies)
        ]
        
        legend_fontsize = 8 
        legend1 = ax_legends.legend(handles=col1_elements, 
                                    title="Stats", title_fontsize=legend_fontsize + 1,
                                    loc='upper left', bbox_to_anchor=(-0.05, 1.0),  
                                    frameon=False, fontsize=legend_fontsize)
        ax_legends.add_artist(legend1) 

        legend2 = ax_legends.legend(handles=col2_elements, 
                                    title="Defense Node Weights", title_fontsize=legend_fontsize + 1,
                                    loc='upper center', bbox_to_anchor=(0.4, 1.0),  
                                    frameon=False, fontsize=legend_fontsize)
        ax_legends.add_artist(legend2)

        legend3 = ax_legends.legend(handles=col3_elements, 
                                    title="Attack Path Weights", title_fontsize=legend_fontsize + 1,
                                    loc='upper right', bbox_to_anchor=(1.05, 1.0), 
                                    frameon=False, fontsize=legend_fontsize)
    # --- End of Legend Section ---
    
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.1) 

    # --- File Saving Section (Keep as is for now) ---
    try:
        # This assumes attack_rate_list and defense_rate_list are defined outside
        for ar in attack_rate_list: 
            for dr in defense_rate_list:
                current_basename = f"{basename}_{ar}_{dr}" 
                if legends:
                    filename = f"basic_{current_basename}_debug.png"
                else:
                    filename = f"basic_{current_basename}.png"
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                print(f"Saved plot as {filename}")

    except NameError:
        print("Warning: attack_rate_list or defense_rate_list not defined. Saving single plot.")
        current_basename = f"{basename}_{attack_rate}_{defense_rate}" 
        if legends:
            filename = f"basic_{current_basename}_debug.png"
        else:
            filename = f"basic_{current_basename}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Saved plot as {filename}")
    # --- End of File Saving Section ---

    plt.show()

In [5]:
graph_plotting(basename)

NameError: name 'basename' is not defined